In [1]:
import os

import sklearn
import pandas as pd
import numpy as np
import cv2
import pickle

In [2]:
from models.darknet53 import darknet_classifier


Using TensorFlow backend.


In [3]:
classes_file_name = './data/classes-trainable.csv'
classes_file = pd.read_csv(classes_file_name)

In [4]:
all_labels = classes_file['label_code']

In [5]:
# set the number of labels which will be used as an output layer size for a model
num_labels = len(all_labels)

# build the index dictionary based on the labels collection
labels_index = {label:idx for idx, label in enumerate(all_labels)}



In [26]:
test_images_dir = './data/stage_1_test_images/'
test_images = os.listdir(test_images_dir)

print (test_images[0])

6b704b475a2b4245336c4d3d.jpg


In [7]:
input_shape = (150, 150, 3)

In [8]:
model = darknet_classifier(input_shape, num_labels)

In [9]:
model.load_weights('model.h5')

/Users/Viktor/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 150, 150, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 150, 150, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 150, 150, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_l

In [11]:
n_channels = 3

In [17]:
def predict(model, image_name, threshold=0.2):
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = np.reshape(image,[1,input_shape[0], input_shape[1], n_channels])
    prediction = model.predict(image)[0]
    
    #print (prediction[prediction > threshold])
    #print (np.sum(prediction))
    
    
    indices = np.argwhere(prediction >= threshold).flatten()
    #print (indices)
    
    labels = all_labels[indices].tolist()
    
    return labels

In [18]:
image_name = test_images_dir + test_images[0]
labels = predict(model, image_name)


print (labels)

['/m/026d34t', '/m/02jvjf', '/m/09j2d', '/m/09j5n']


In [27]:
submission = {'image_id':[], 'labels':[]}

for index, img in enumerate(test_images):
    if index > 100:
        break
    
    image_name = test_images_dir + img
    
    print (image_name)
    
    image_id = img[:-4]
    
    labels = predict(model, image_name)
    
    submission['image_id'].append(image_id)
    submission['labels'].append(labels)
    
submission = pd.DataFrame(submission)

submission.head()

submission.to_csv('darknet53_150x150_2epochs.csv')
    



./data/stage_1_test_images/6b704b475a2b4245336c4d3d.jpg
./data/stage_1_test_images/357969376c73486c7a68453d.jpg
./data/stage_1_test_images/31677368764467784869413d.jpg
./data/stage_1_test_images/356d6347746834374a776f3d.jpg
./data/stage_1_test_images/5a517565354c573837564d3d.jpg
./data/stage_1_test_images/6e46457854654876524b6f3d.jpg
./data/stage_1_test_images/79523153466c48707855343d.jpg
./data/stage_1_test_images/68555a4f33725a696e65513d.jpg
./data/stage_1_test_images/536b5354394170655345513d.jpg
./data/stage_1_test_images/553030375638326c5064493d.jpg
./data/stage_1_test_images/4a574c6c484861655179413d.jpg
./data/stage_1_test_images/5639305a4b4c6c6f4d65593d.jpg
./data/stage_1_test_images/326966764f626c6e384d673d.jpg
./data/stage_1_test_images/656f64515a5855796e47453d.jpg
./data/stage_1_test_images/436c7a4771306a627756383d.jpg
./data/stage_1_test_images/43397650392b4f6e5157493d.jpg
./data/stage_1_test_images/576c474e4c3979653431593d.jpg
./data/stage_1_test_images/696755444f376b564f2f4